In [1]:
import numpy as np

# Quadratic Discriminant Analysis (QDA) - Explication détaillée

Le QDA est un algorithme de classification supervisée qui repose sur l’hypothèse que les données de chaque classe suivent une distribution gaussienne multivariée, mais avec des matrices de covariance distinctes pour chaque classe. Voici une décomposition étape par étape.

---

## Étape 1 : Initialisation du modèle
### Objectif :
Créer une classe `QDA` pour gérer les données nécessaires à l'entraînement et à la prédiction.

In [34]:
class QDA:
    def __init__(self):
        self.classes_ = None          # Liste des classes uniques
        self.means_ = {}              # Moyenne des données pour chaque classe
        self.covariances_ = {}        # Matrice de covariance pour chaque classe
        self.priors_ = {}             # Probabilités a priori des classes


- **`classes_`** : Contiendra les classes uniques dans les données d'entraînement.  
- **`means_`** : Stocke les vecteurs de moyennes pour chaque classe.  
- **`covariances_`** : Stocke les matrices de covariance pour chaque classe.  
- **`priors_`** : Contient les probabilités a priori de chaque classe (fréquence relative dans les données).


## Étape 2 : Entraîner le modèle (`fit`)
### Objectif :
Calculer les paramètres nécessaires pour chaque classe à partir des données d’entraînement :  
- Moyennes
- Matrices de covariance
- Probabilités a priori

In [35]:
def fit(self, X, y):
    self.classes_ = np.unique(y)  # Trouver toutes les classes
    for cls in self.classes_:
        X_cls = X[y == cls]                           # Sélectionner les données de la classe
        self.means_[cls] = np.mean(X_cls, axis=0)     # Moyenne des données
        self.covariances_[cls] = np.cov(X_cls, rowvar=False)  # Matrice de covariance
        self.priors_[cls] = X_cls.shape[0] / X.shape[0]       # Proportion des données pour la classe


- **Étapes principales** :  
  1. Identifier les classes avec une méthode pour trouver les valeurs uniques dans les étiquettes.  
  2. Pour chaque classe :  
     - Filtrer les observations appartenant à cette classe.
     - Calculer la **moyenne** des observations.
     - Calculer la **matrice de covariance** (représente les dépendances entre les variables).  
     - Calculer la probabilité a priori comme le ratio entre le nombre d'observations de cette classe et le nombre total d'observations.  


## Étape 3 : Calculer la densité gaussienne multivariée (`_gaussian_density`)
### Objectif :
Définir une fonction pour calculer la densité de probabilité d’un vecteur dans une distribution gaussienne multivariée.

#### Formule utilisée :
$$
P(x | y = k) = \frac{1}{\sqrt{(2\pi)^d |\Sigma|}} \exp\left(-\frac{1}{2}(x - \mu)^T \Sigma^{-1} (x - \mu)\right)
$$

- $\mu$  : vecteur de moyenne (par classe)  
- $\Sigma$ : matrice de covariance (par classe)  
- $|\Sigma|$ : déterminant de la matrice de covariance  
- $\Sigma^{-1}$ : inverse de la matrice de covariance  


In [36]:
def _gaussian_density(self, x, mean, covariance):
    size = len(mean)  # Nombre de dimensions
    epsilon = 1e-6    # Régularisation pour éviter une matrice singulière
    covariance += np.eye(size) * epsilon

    det = np.linalg.det(covariance)   # Déterminant de la matrice de covariance
    inv_cov = np.linalg.inv(covariance)  # Inverse de la matri  ce de covariance

    # Normalisation et exponentielle
    norm_const = 1.0 / (np.power(2 * np.pi, size / 2) * np.sqrt(det))
    x_diff = x - mean
    return norm_const * np.exp(-0.5 * x_diff @ inv_cov @ x_diff.T)


- **Régularisation** : Ajout d’un petit terme à la diagonale de la matrice de covariance pour éviter des problèmes numériques si la matrice est singulière.
- **Densité** : Calcule la probabilité d'une observation sous la distribution gaussienne définie par la moyenne et la matrice de covariance.

## Étape 4 : Prédire la classe pour de nouvelles données (`predict`)
### Objectif :
Pour chaque observation, calculer la probabilité pour chaque classe, puis assigner la classe avec la probabilité maximale.

In [37]:
def predict(self, X):
    predictions = []
    for x in X:
        class_probs = {}
        for cls in self.classes_:
            # Probabilité conditionnelle pour chaque classe
            likelihood = self._gaussian_density(x, self.means_[cls], self.covariances_[cls])
            class_probs[cls] = likelihood * self.priors_[cls]  # Bayes: P(y|x) ∝ P(x|y)P(y)
        # Classe avec la probabilité maximale
        predictions.append(max(class_probs, key=class_probs.get))
    return np.array(predictions)

- **Étapes principales** :  
  1. Pour chaque observation dans les données de test :  
     - Calculer $P(x | y = k) \cdot P(y = k)$ pour chaque classe $ k$.  
  2. Choisir la classe $ k$ qui maximise cette valeur.  
  3. Ajouter la prédiction à la liste des résultats.  

## Résumé
### Étapes principales du QDA :
1. Calculer les **paramètres des distributions gaussiennes** pour chaque classe :
   - Moyenne ($ \mu $).
   - Matrice de covariance ($ \Sigma $).
   - Probabilité a priori ($ P(y) $).

2. Pour une observation donnée, estimer la **probabilité conditionnelle** pour chaque classe en appliquant la formule de densité gaussienne multivariée.

3. **Classer** l’observation dans la classe ayant la probabilité maximale.

---

## Points importants
- **Régularisation** : Nécessaire pour gérer les matrices singulières.  
- **Normalité** : Le QDA repose sur l’hypothèse que les données suivent une distribution normale multivariée. Si ce n’est pas le cas, les résultats peuvent être sous-optimaux.  
- **Complexité** : La matrice de covariance est inversée pour chaque classe, ce qui peut être coûteux pour des ensembles de données à forte dimension.  